# Building a ChatBot with Deep NLP

### Importing the libraries

In [1]:
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.compat.v1.disable_v2_behavior()
import tensorflow_addons as tfa
import tf_slim as tfs
import re
import time

Instructions for updating:
non-resource variables are not supported in the long term


/Users/danielzajkowski/.pyenv/versions/3.10.9/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## Data preprocessing

### Importing the dataset

In [2]:
lines = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

### Creating a dictionary that maps each line and its id

In [3]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

### Creating a list of all of the conversations

In [4]:
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
    conversations_ids.append(_conversation.split(','))

### Getting separately the questions and the answers

In [5]:
questions = []
answers = []
for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

### Doing a first cleaning of the texts

In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

### Cleaning the questions

In [7]:
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))

### Cleaning the answers

In [8]:
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

### Creating a dictionary that maps each word to its number of occurences

In [9]:
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

### Creating two dictionaries that map the questions words and the answers words to a unique integer

In [10]:
threshold = 20
questionswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold:
        questionswords2int[word] = word_number
        word_number += 1

answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold:
        answerswords2int[word] = word_number
        word_number += 1

### Adding the last tokens to these two dictionaries

In [11]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
for token in tokens:
    answerswords2int[token] = len(answerswords2int) + 1

### Creating the inverse dictionary of the answerswords2int dictionary

In [12]:
answersints2word = {w_i: w for w, w_i in answerswords2int.items()}

### Adding the End Of String token to the end of every answer

In [13]:
for i in range(len(clean_answers)):
    clean_answers[i] += ' <EOS>'

### Translating all the questions and the answers into integers and Replacing all the words that were filtered out by < OUT >

In [14]:
questions_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_into_int.append(ints)

answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_into_int.append(ints)

### Sorting questions and answers by the lenght of questions

In [15]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range (1, 25 + 1):
    for i in enumerate(questions_into_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])

## Building the Seq2Seq Model

### Creating placeholder for the inputs and the targets

In [16]:
def model_inputs():
    inputs = tf.compat.v1.placeholder(tf.int32, [None, None], name='input')
    targets = tf.compat.v1.placeholder(tf.int32, [None, None], name='target')
    lr = tf.compat.v1.placeholder(tf.float32, [None, None], name='learning_rate')
    keep_prob = tf.compat.v1.placeholder(tf.float32, name='keep_prob')
    return inputs, targets, lr, keep_prob

### Preprocessing the targets

In [17]:
def preprocess_targets(targets, word2int, batch_size):
    left_side = tf.fill([batch_size, 1], word2int['<SOS>'])
    right_side = tf.strided_slice(targets, [0, 0], [batch_size, -1], [1, 1])
    preprocess_targets = tf.concat([left_side, right_side], 1)
    return preprocess_targets

### Creating the Encoder RNN Layer

In [46]:
def encoder_rnn(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    lstm = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.compat.v1.nn.rnn_cell.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    encoder_cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([lstm_dropout] * num_layers)
    encoder_output, encoder_state = tf.compat.v1.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell, cell_bw = encoder_cell, sequence_length = sequence_length, inputs = rnn_inputs, dtype = tf.float32)
    return encoder_state

### Decoding the training set

In [19]:
def decode_training_set(encoder_state, decoder_cell, decoder_embedded_input, sequence_length, decoding_scope, output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tfa.seq2seq.AttentionWrapper(attention_states, attention_option="bahdanau", num_units=decoder_cell.output_size)
    training_decoder_function = tfa.seq2seq.BasicDecoder(encoder_state[0], attention_keys, attention_values, attention_score_function, attention_construct_function, name="attn_dec_train")
    decoder_output, decoder_final_state, decoder_final_context_state = tfa.seq2seq.dynamic_decode(decoder_cell, training_decoder_function, decoder_embedded_input, sequence_length, scope=decoding_scope)
    decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prob)
    return output_function(decoder_output_dropout)

### Decoding the test/validation set

In [20]:
def decode_test_set(encoder_state, decoder_cell, decoder_embeddings_matrix_input, sos_id, eos_id, maximum_length, num_words, decoding_scope, output_function, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tfa.seq2seq.AttentionWrapper(attention_states, attention_option="bahdanau", num_units=decoder_cell.output_size)
    test_decoder_function = tfa.seq2seq.BasicDecoder(output_function, encoder_state[0], attention_keys, attention_values, attention_score_function, attention_construct_function, decoder_embeddings_matrix_input, sos_id, eos_id, maximum_length, num_words, name="attn_dec_inf")
    test_predictions, _, _ = tfa.seq2seq.dynamic_decode(decoder_cell, test_decoder_function, scope=decoding_scope)
    return test_predictions

### Creating the Decoder RNN

In [21]:
def decoder_rnn(decoder_embedded_input, decoder_embeddings_matrix, encoder_state, num_words, sequence_length, rnn_size, num_layers, word2int, keep_prob, batch_size):
    with tf.compat.v1.variable_scope("decoding") as decoding_scope:
        lstm = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(rnn_size)
        lstm_dropout = tf.compat.v1.nn.rnn_cell.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        decoder_cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([lstm_dropout] * num_layers)
        weights = tf.compat.v1.truncated_normal_initializer(stddev = 0.1)
        biases = tf.zeros_initializer()
        output_function = lambda x: tfs.fully_connected(x, num_words, None, scope = decoding_scope, weights_initializer = weights, biases_initializer = biases)
        training_predictions = decode_training_set(encoder_state, decoder_cell, decoder_embedded_input, sequence_length, output_function, keep_prob, batch_size)
        decoding_scope.reuse_variables()
        test_predictions = decode_test_set(encoder_state, decoder_cell, decoder_embeddings_matrix, word2int['<SOS>'], word2int['<EOS>'], sequence_length - 1, num_words, decoding_scope, output_function, keep_prob, batch_size)
    return training_predictions, test_predictions

### Building the seq2seq model

In [51]:
# inputs -- question ask to the chatbot
# target -- answer
# answers_num_words -- total number of words in all answers
# encoder_embedding_size -- number of dimesnison in the embbeding matrix
def seq2seq_model(inputs, targets, keep_prob, batch_size, sequence_length, answers_num_words, questions_num_words, encoder_embedding_size, decoder_embedding_size, rnn_size, num_layers, questionswords2int):
    encoder_embedded_input = tf.keras.layers.Embedding(answers_num_words + 1, encoder_embedding_size, embeddings_initializer = tf.random_uniform_initializer(0,1))
    encoder_state = encoder_rnn(encoder_embedded_input, rnn_size, num_layers, keep_prob, sequence_length)
    preprocesed_targets = preprocess_targets(targets, questionswords2int, batch_size)
    decoder_embeddings_matrix = tf.Variable(tf.compat.v1.random_uniform([questions_num_words + 1, decoder_embedding_size],0, 1))
    decoder_embedded_input = tf.nn.embedding_lookup(decoder_embeddings_matrix, preprocesed_targets)
    training_predictions, test_predictions = decoder_rnn(decoder_embedded_input, decoder_embeddings_matrix, encoder_state, questions_num_words, sequence_length, rnn_size, num_layers, questionswords2int, keep_prob, batch_size)
    return training_predictions, test_predictions

## Training the Seq2Seq Model

### Setting the Hyperparameters

In [23]:
epochs = 100
batch_size = 64
rnn_size = 512
num_layers = 3
encoding_embedding_size = 512 
decoding_embedding_size = 512
learning_rate = 0.01
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.5

### Defining a session

In [24]:
tf.compat.v1.reset_default_graph()
tf.compat.v1.InteractiveSession()

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



### Loading the model inputs

In [25]:
inputs, targets, lr, keep_prob = model_inputs()

### Setting the sequence length

In [26]:
sequence_length = tf.compat.v1.placeholder_with_default(25, None, name = 'sequence_length')

### Getting the shape of the inputs tensor

In [27]:
input_shape = tf.shape(inputs)

### Getting the training and test predictions

In [52]:
training_predictions, test_predictions = seq2seq_model(tf.reverse(inputs, [-1]), targets, keep_prob, batch_size, sequence_length, len(answerswords2int), len(questionswords2int), encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, questionswords2int)

/var/folders/_k/56ln05tx4vs6_gx2ht0p_pzc0000gn/T/ipykernel_91566/677993718.py:2: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  lstm = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(rnn_size)


TypeError: Failed to convert elements of <keras.layers.core.embedding.Embedding object at 0x2d7a03760> to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.